In [1]:
import matplotlib

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division

Precisamos de algumas funções especializadas do NLTK que não estão incluídas por padrão. É possível baixar apenas a parte com as "stopwords", palavras irrelevantes, mas talvez seja mais fácil baixar tudo no NLTK. Observe que é um processo muito demorado; levou mais de 30 minutos em meu computador.

In [2]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     C:\Us

[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     C:\Users\Atelli\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |    

True

Baixar o pacote "stopwords" do NLTK.

In [3]:
from nltk.corpus import stopwords

## Ler dados

In [27]:
with codecs.open("relatorio.txt", "r", encoding="utf-8") as f:
    relatorio_1 = f.read()
with codecs.open("coronavac.txt", "r", encoding="utf-8") as f:
    coronavac_1 = f.read()

#Processar dados
#Verificar palavras irrelevantes em português.

In [28]:
esw = stopwords.words('portuguese')
esw.append("would")

Filtrar tokens (usando expressões regulares).

In [29]:
word_pattern = re.compile("^\w+$")

Criar função para contagem de tokens.

In [30]:
def get_text_counter(text):
    tokens = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens = list(map(lambda x: x.lower(), tokens))
    tokens = [token for token in tokens if re.match(word_pattern, token) and token not in esw]
    return collections.Counter(tokens), len(tokens)

Criar função para cálculo da frequência absoluta e da frequência relativa das palavras mais comuns.

In [31]:
def make_df(counter, size):
    abs_freq = np.array([el[1] for el in counter])
    rel_freq = abs_freq / size
    index = [el[0] for el in counter]
    df = pd.DataFrame(data=np.array([abs_freq, rel_freq]).T, index=index, columns=["Frequencia Absoluta", "Frequencia Relativa"])
    df.index.name = "Palavras mais comuns"
    return df

## Analisar textos individuais

Calcular as palavras mais comuns de _Relatorio_Anvisa_. Isso demora um pouco. Então, exibir as 10 mais comuns.

In [41]:
re_counter, re_size = get_text_counter(relatorio_1)
make_df(re_counter.most_common(10), re_size)

,Frequencia Absoluta,Frequencia Relativa
Palavras mais comuns,,
vacina,81.0,0.017020
19,67.0,0.014079
covid,56.0,0.011767
2020,52.0,0.010927
uso,51.0,0.010717
vacinas,51.0,0.010717
4,45.0,0.009456
anvisa,45.0,0.009456
eficácia,43.0,0.009036


Salvar as 1.000 palavras mais comuns de _Relatorio_ como CSV.

In [42]:
re_df = make_df(re_counter.most_common(1000), re_size)
re_df.to_csv("RE_1000.csv")

In [43]:
co_counter, co_size = get_text_counter(coronavac_1)
make_df(co_counter.most_common(10), co_size)

,Frequencia Absoluta,Frequencia Relativa
Palavras mais comuns,,
estudo,42.0,0.021298
eficácia,36.0,0.018256
vacina,34.0,0.017241
covid,27.0,0.013692
19,25.0,0.012677
2,21.0,0.010649
anos,19.0,0.009635
dose,18.0,0.009128
0,18.0,0.009128


In [44]:
co_df = make_df(co_counter.most_common(1000), co_size)
co_df.to_csv("co_1000.csv")

## Comparar textos

Identificar as palavras mais comuns nos dois documentos.

In [45]:
all_counter = co_counter + re_counter
all_df = make_df(co_counter.most_common(1000), 1)
most_common_words = all_df.index.values

Criar um quadro de dados com as diferenças de frequência das palavras.

In [46]:
df_data = []
for word in most_common_words:
    re_c = re_counter.get(word, 0) / re_size
    co_c = co_counter.get(word, 0) / co_size
    d = abs(re_c - co_c)
    df_data.append([re_c, co_c, d])
dist_df = pd.DataFrame(data=df_data, index=most_common_words,
                       columns=["Relatorio Anvisa Frequência Relativa", "Apresentação Coranavac Frequência Relativa",
                                "Diferença de Frequência Relativa"])
dist_df.index.name = "Palavras mais comuns"
dist_df.sort_values("Diferença de Frequência Relativa", ascending=False, inplace=True)

Exibir as palavras mais distintas.

In [47]:
dist_df.head(10)

,Relatorio Anvisa Frequência Relativa,Apresentação Coranavac Frequência Relativa,Diferença de Frequência Relativa
Palavras mais comuns,,,
estudo,0.001261,0.021298,0.020037
2020,0.010927,0.001014,0.009912
vacinas,0.010717,0.001014,0.009702
eficácia,0.009036,0.018256,0.009220
anos,0.000841,0.009635,0.008794
0,0.000420,0.009128,0.008708
dose,0.000630,0.009128,0.008497
anvisa,0.009456,0.002028,0.007427
idade,0.000420,0.007099,0.006679


Salvar a lista completa com as palavras distintas como um CSV intitulado "distintas.csv".

In [48]:
dist_df.to_csv("distintas.csv")